# **Ensemble of Local Model-Agnostic Explanation Models for Robust Local Feature Importance Ranking**

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import numpy as np
np.set_printoptions(suppress=True)
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, roc_auc_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import NearestNeighbors

# Explainable AI tools:
import shap
from lime.lime_tabular import LimeTabularExplainer
from alibi.explainers import AnchorTabular # why not used the original anchor package?

from scipy.stats import spearmanr, pearsonr

from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam  # Import the Adam optimizer

from tools.topsis import Topsis

2024-11-06 21:50:26.781029: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-11-06 21:50:26.922519: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1730940626.976185    5908 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1730940626.992950    5908 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-06 21:50:27.127441: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

In [3]:
# Configure pandas output
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 40)

# Data Loading and Preprocessing

### Data loading and summarizing

In [4]:
original_data = pd.read_csv('german_credit_data_updated.csv')

# Dataset overview - German Credit Risk (from Kaggle):
# 1. Age (numeric)
# 2. Sex (text: male, female)
# 3. Job (numeric: 0 - unskilled and non-resident, 1 - unskilled and resident, 2 - skilled, 3 - highly skilled)
# 4. Housing (text: own, rent, or free)
# 5. Saving accounts (text - little, moderate, quite rich, rich)
# 6. Checking account (numeric, in DM - Deutsch Mark)
# 7. Credit amount (numeric, in DM)
# 8. Duration (numeric, in month)
# 9. Purpose (text: car, furniture/equipment, radio/TV, domestic appliances, repairs, education, business, vacation/others)

display(original_data.head())
display(original_data.describe())
display(original_data.info())

# Display the unique values of thprecision=3, e categorical features:
print('Unique values of the categorical features:')
for col in original_data.select_dtypes(include='object'):
    print(f'\t- {col}: {original_data[col].unique()}')

,Unnamed: 0,Age,Sex,Job,Housing,Saving accounts,Checking account,Credit amount,Duration,Purpose,Credit Risk
0,0,67,male,2,own,NaN,little,1169,6,radio/TV,1
1,1,22,female,2,own,little,moderate,5951,48,radio/TV,2
2,2,49,male,1,own,little,NaN,2096,12,education,1
3,3,45,male,2,free,little,little,7882,42,furniture/equipment,1
4,4,53,male,2,free,little,little,4870,24,car,2


,Unnamed: 0,Age,Job,Credit amount,Duration,Credit Risk
count,954.000000,954.000000,954.000000,954.000000,954.000000,954.000000
mean,476.500000,35.501048,1.909853,3279.112159,20.780922,1.302935
std,275.540378,11.379668,0.649681,2853.315158,12.046483,0.459768
min,0.000000,19.000000,0.000000,250.000000,4.000000,1.000000
25%,238.250000,27.000000,2.000000,1360.250000,12.000000,1.000000
50%,476.500000,33.000000,2.000000,2302.500000,18.000000,1.000000
75%,714.750000,42.000000,2.000000,3975.250000,24.000000,2.000000
max,953.000000,75.000000,3.000000,18424.000000,72.000000,2.000000


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 954 entries, 0 to 953
Data columns (total 11 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Unnamed: 0        954 non-null    int64 
 1   Age               954 non-null    int64 
 2   Sex               954 non-null    object
 3   Job               954 non-null    int64 
 4   Housing           954 non-null    object
 5   Saving accounts   779 non-null    object
 6   Checking account  576 non-null    object
 7   Credit amount     954 non-null    int64 
 8   Duration          954 non-null    int64 
 9   Purpose           954 non-null    object
 10  Credit Risk       954 non-null    int64 
dtypes: int64(6), object(5)
memory usage: 82.1+ KB


None

Unique values of the categorical features:
	- Sex: ['male' 'female']
	- Housing: ['own' 'free' 'rent']
	- Saving accounts: [nan 'little' 'quite rich' 'rich' 'moderate']
	- Checking account: ['little' 'moderate' nan 'rich']
	- Purpose: ['radio/TV' 'education' 'furniture/equipment' 'car' 'business'
 'domestic appliances' 'repairs' 'vacation/others']


### Data preprocessing

In [5]:
preprocessed_data = original_data.copy()

# For savings and checking accounts, we will replace the missing values with 'none':
preprocessed_data['Saving accounts'].fillna('none', inplace=True)
preprocessed_data['Checking account'].fillna('none', inplace=True)

# Dropping index column:
preprocessed_data.drop(columns=['Unnamed: 0'], inplace=True)

# Using pd.dummies to one-hot-encode the categorical features
preprocessed_data["Job"] = preprocessed_data["Job"].map({0: 'unskilled_nonresident', 1: 'unskilled_resident',
                                                         2: 'skilled', 3: 'highlyskilled'})

categorical_features = preprocessed_data.select_dtypes(include='object').columns
numerical_features = preprocessed_data.select_dtypes(include='number').columns.drop('Credit Risk')
print(f'Categorical features: {categorical_features}')
print(f'Numerical features: {numerical_features}')

preprocessed_data = pd.get_dummies(preprocessed_data, columns=categorical_features, dtype='int64')

# Remapping the target variable to 0 and 1:
preprocessed_data['Credit Risk'] = preprocessed_data['Credit Risk'].map({1: 0, 2: 1})

# Make sure all column names are valid python identifiers (important for pd.query() calls):
preprocessed_data.columns = preprocessed_data.columns.str.replace(' ', '_')
preprocessed_data.columns = preprocessed_data.columns.str.replace('/', '_')

# Normalizing the data
scaler = StandardScaler()
scaled_preprocessed_data = scaler.fit_transform(preprocessed_data)

display(preprocessed_data.head())
display(preprocessed_data.info())

display(scaled_preprocessed_data)


Categorical features: Index(['Sex', 'Job', 'Housing', 'Saving accounts', 'Checking account',
       'Purpose'],
      dtype='object')
Numerical features: Index(['Age', 'Credit amount', 'Duration'], dtype='object')


,Age,Credit_amount,Duration,Credit_Risk,Sex_female,Sex_male,Job_highlyskilled,Job_skilled,Job_unskilled_nonresident,Job_unskilled_resident,Housing_free,Housing_own,Housing_rent,Saving_accounts_little,Saving_accounts_moderate,Saving_accounts_none,Saving_accounts_quite_rich,Saving_accounts_rich,Checking_account_little,Checking_account_moderate,Checking_account_none,Checking_account_rich,Purpose_business,Purpose_car,Purpose_domestic_appliances,Purpose_education,Purpose_furniture_equipment,Purpose_radio_TV,Purpose_repairs,Purpose_vacation_others
0,67,1169,6,0,0,1,0,1,0,0,0,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,0
1,22,5951,48,1,1,0,0,1,0,0,0,1,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0
2,49,2096,12,0,0,1,0,0,0,1,0,1,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0
3,45,7882,42,0,0,1,0,1,0,0,1,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0
4,53,4870,24,1,0,1,0,1,0,0,1,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 954 entries, 0 to 953
Data columns (total 30 columns):
 #   Column                       Non-Null Count  Dtype
---  ------                       --------------  -----
 0   Age                          954 non-null    int64
 1   Credit_amount                954 non-null    int64
 2   Duration                     954 non-null    int64
 3   Credit_Risk                  954 non-null    int64
 4   Sex_female                   954 non-null    int64
 5   Sex_male                     954 non-null    int64
 6   Job_highlyskilled            954 non-null    int64
 7   Job_skilled                  954 non-null    int64
 8   Job_unskilled_nonresident    954 non-null    int64
 9   Job_unskilled_resident       954 non-null    int64
 10  Housing_free                 954 non-null    int64
 11  Housing_own                  954 non-null    int64
 12  Housing_rent                 954 non-null    int64
 13  Saving_accounts_little       954 non-null    int64

None

array([[ 2.7694545 , -0.7399179 , -1.22763429, ...,  1.62518349,
        -0.14633276, -0.11286653],
       [-1.18704073,  0.93690642,  2.26068929, ...,  1.62518349,
        -0.14633276, -0.11286653],
       [ 1.18685641, -0.41486224, -0.72930235, ..., -0.61531514,
        -0.14633276, -0.11286653],
       ...,
       [-1.0111965 , -0.39768023,  1.26402541, ..., -0.61531514,
        -0.14633276, -0.11286653],
       [-0.65950803,  0.29240557,  0.26736153, ..., -0.61531514,
        -0.14633276, -0.11286653],
       [-0.83535227,  2.69823821,  1.26402541, ..., -0.61531514,
        -0.14633276, -0.11286653]])

### Splitting the data into training and testing sets:

In [6]:
y = preprocessed_data['Credit_Risk']
X = preprocessed_data.drop(columns='Credit_Risk')

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Sklearn Model Fitting

In [7]:
clf = RandomForestClassifier(random_state=42)
clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)

print(f'Accuracy: {accuracy_score(y_test, y_pred)}')
print(f'ROC AUC: {roc_auc_score(y_test, y_pred)}')

Accuracy: 0.7696335078534031
ROC AUC: 0.6830357142857143


# Algorithm Implementation
Given a dataset, an ML model trained on this dataset (whose predictions we want to explain), and a set of explanation tools from which the aggregate explanation model will be built:

1. Apply each explanation model to the instance whose prediction is to be explained, obtaining a feature importance ranking for each explanation model.
2. Generate a variation of the original dataset by introducing noise.
3. Repeat step 1 on this noisy dataset to obtain a new set of feature importance rankings.
4. Assess the robustness of each explanation model by comparing its feature importance ranking on the original data with the ranking on the noisy data.
5. Finally, compute an aggregate explanation ranking by taking the weighted average of the rankings obtained in step 1, with the weights based on each model's stability (determined in step 4).

### Step 1
Apply each explanation model to the instance whose prediction is to be explained, obtaining a feature importance ranking for each explanation model.

In [8]:
class FeatureImportanceCalculator():
    def __init__(self, clf, X_train: pd.DataFrame | np.ndarray, predict_proba_function: callable = None):
        self.X_train = X_train
        self.clf = clf
        
        if predict_proba_function is not None:
            self.predict_proba_function = predict_proba_function
        elif hasattr(clf, 'predict_proba') and predict_proba_function is None:
            self.predict_proba_function = clf.predict_proba
        else:
            raise ValueError('The classifier does not have a predict_proba method and no predict_proba_function was provided.')
        
        self.anchor_explainer = AnchorTabular(predictor=self.predict_proba_function, feature_names=self.X_train.columns) # TODO: fix parameters
        self.anchor_explainer.fit(self.X_train.values)


    def get_lime_ranking(self, instance_data_row) -> pd.DataFrame:
        """
        Returns a DataFrame with the feature importance ranking using LIME, ordered by abs(importance).
        """
        
        explainer = LimeTabularExplainer(self.X_train.values, feature_names=self.X_train.columns, discretize_continuous=False)
        lime_exp = explainer.explain_instance(instance_data_row, self.predict_proba_function, num_features=len(self.X_train.columns))
        
        ranking = pd.DataFrame(lime_exp.as_list(), columns=['feature', 'score'])

        return ranking
    
    def get_shap_ranking(self, instance_data_row, explainer_type: shap.Explainer = shap.KernelExplainer, **additional_explainer_args) -> pd.DataFrame:
        """
        Returns a DataFrame with the feature importance ranking using SHAP, ordered by abs(importance).
        """
        explainer = explainer_type(self.clf, self.X_train, **additional_explainer_args)
        shap_values = explainer.shap_values(instance_data_row)

        ranking = pd.DataFrame(list(zip(self.X_train.columns, shap_values[:, 0])), columns=['feature', 'score'])
        ranking = ranking.sort_values(by='score', ascending=False, key=lambda x: abs(x)).reset_index(drop=True)
        
        return ranking
    
    def get_anchor_ranking(self, instance_data_row: pd.Series | np.ndarray) -> pd.DataFrame:
        """
        Returns a DataFrame with the feature importance ranking using Anchor, ordered by abs(importance).
        Feature importance is not directly available in the AnchorTabular class. In order to obtain it, we can
        calculate the percentage of rows in the training data that are not covered by the anchor rule. The more
        rows that are not covered, the more important the feature is.
        """

        if isinstance(instance_data_row, pd.Series):
            instance_data_row = instance_data_row.to_numpy()

        feature_importances = {feature: 0 for feature in self.X_train.columns}
        explanation = self.anchor_explainer.explain(instance_data_row)
        
        for rule in explanation.anchor:
            # Extract the feature name from the rule string
            # This method won't work for column names that have spaces in them or that don't contain any letters
            for expression_element in rule.split():
                if any(c.isalpha() for c in expression_element):
                    referenced_feature = expression_element
                    break

            rule_coverage = X_train.query(rule).shape[0] / X_train.shape[0]
            feature_importances[referenced_feature] = 1 - rule_coverage
        
        return pd.DataFrame(list(feature_importances.items()), columns=['feature', 'score']).sort_values(by='score', ascending=False).reset_index(drop=True)

In [9]:
# Example usage:
feature_importances = FeatureImportanceCalculator(clf, X_train)
    
sample_idx = 0
lime_ranking = feature_importances.get_lime_ranking(X_test.iloc[sample_idx])
shap_ranking = feature_importances.get_shap_ranking(X_test.iloc[sample_idx], explainer_type=shap.TreeExplainer)
anchor_ranking = feature_importances.get_anchor_ranking(X_test.iloc[sample_idx])

print('LIME ranking:')
display(lime_ranking)
print('SHAP ranking:')
display(shap_ranking)
print('Anchor ranking:')
display(anchor_ranking)

LIME ranking:


,feature,score
0,Checking_account_none,-0.059521
1,Duration,0.055533
2,Checking_account_little,0.036489
3,Age,-0.027242
4,Checking_account_moderate,0.017293
5,Housing_own,-0.013181
6,Purpose_radio_TV,-0.012192
7,Sex_male,-0.010750
8,Credit_amount,0.008273
9,Sex_female,0.007927


SHAP ranking:


,feature,score
0,Duration,0.051965
1,Checking_account_none,-0.048818
2,Age,0.044427
3,Checking_account_little,0.030740
4,Checking_account_moderate,-0.025005
5,Credit_amount,0.018809
6,Saving_accounts_moderate,0.011132
7,Purpose_furniture_equipment,0.009065
8,Sex_female,0.007021
9,Purpose_car,0.006882


Anchor ranking:


,feature,score
0,Purpose_furniture_equipment,0.813893
1,Age,0.503277
2,Duration,0.433814
3,Checking_account_little,0.283093
4,Saving_accounts_quite_rich,0.000000
5,Purpose_repairs,0.000000
6,Purpose_radio_TV,0.000000
7,Purpose_education,0.000000
8,Purpose_domestic_appliances,0.000000
9,Purpose_car,0.000000


### Step 2
Generate a variation of the original dataset by introducing noise.

In [10]:
def get_noisy_data_autoencoder(X: pd.DataFrame, categorical_features_names: list[str], encoding_dim: int = 5, num_features_to_replace: int = 2, epochs=500) -> pd.DataFrame:
    """
    Returns a DataFrame containing a noisy variation of the data.

    The noise is generated by swapping the values of a small number of features between a sample and a random close neighbor.
    To determine the neighbors, we use an autoencoder to reduce the dimensionality of the data and then calculate the use the NearestNeightbors algorithm in the reduced space.
    """

    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)
    
    input_dim = X_scaled.shape[1]

    input_layer = Input(shape=(input_dim,))
    encoded = Dense(encoding_dim, activation='relu')(input_layer)
    decoded = Dense(input_dim, activation='sigmoid')(encoded)

    autoencoder = Model(inputs=input_layer, outputs=decoded)
    encoder = Model(inputs=input_layer, outputs=encoded)
    autoencoder.compile(optimizer=Adam(), loss='mean_squared_error')
    autoencoder.fit(X_scaled, X_scaled, epochs=epochs, batch_size=32, shuffle=True, validation_split=0.2)
    # Extract hidden layer representation:
    hidden_representation = encoder.predict(X_scaled)

    # Compute Nearest Neighbors using hidden_representation
    nbrs = NearestNeighbors(n_neighbors=5, algorithm='auto').fit(hidden_representation) # TODO: here, hidden_representation is just the autoencoder fit to the scaled X data; see if this is the way to do this
    distances, indices = nbrs.kneighbors(hidden_representation)

    X_noisy = X.copy()

    # Get id's of columns that belong to the same categorical feature (after being one-hot-encodeded);
    # Columns that belong to the same categorical feature start with the same name, and will be treated as a single feature when adding noise.
    categorical_features_indices = [
        [X.columns.get_loc(col_name) for col_name in X.columns if col_name.startswith(feature)]
        for feature in categorical_features_names
    ]

    # Replace features with random neighbor's features
    for i in range(X.shape[0]):  # Iterate over each sample
        available_features_to_replace = list(range(X.shape[1]))
        for j in range(num_features_to_replace):
            # Select features to replace; if the feture selected belong to one of the lists in categorical_features_indices, we will replace all the features in that list
            features_to_replace = np.random.choice(available_features_to_replace, 1)
            for feature_indices in categorical_features_indices:
                if features_to_replace in feature_indices:
                    features_to_replace = feature_indices
                    break
            
            # Remove the selected features from the list of available features to replace
            available_features_to_replace = [f for f in available_features_to_replace if f not in features_to_replace]

            # Choose a random neighbor from the nearest neighbors
            neighbor_idx = np.random.choice(indices[i][1:])

            # Replace the selected features with the neighbor's features
            X_noisy.iloc[i, features_to_replace] = X.iloc[neighbor_idx, features_to_replace]

    return X_noisy

In [11]:
# Example of usage:
display(X.shape)
noisy_data = get_noisy_data_autoencoder(X, categorical_features_names=categorical_features, encoding_dim=5, num_features_to_replace=2, epochs=10)

# Split the noisy data the same way as the original data
# X_train_noisy, X_test_noisy = train_test_split(noisy_data, test_size=0.2, random_state=41) # Split it with the same random_state as the original data
X_train_noisy = noisy_data.loc[X_train.index]
X_test_noisy = noisy_data.loc[X_test.index]
display("Mean Absolute Difference: ", np.mean(np.abs(X - noisy_data)))

(954, 29)

Epoch 1/10


W0000 00:00:1730940630.066251    5908 gpu_device.cc:2344] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 1.2961 - val_loss: 1.2185
Epoch 2/10
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1.2382 - val_loss: 1.1980
Epoch 3/10
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1.2097 - val_loss: 1.1788
Epoch 4/10
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 1.2249 - val_loss: 1.1603
Epoch 5/10
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 1.1889 - val_loss: 1.1420
Epoch 6/10
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 1.1696 - val_loss: 1.1237
Epoch 7/10
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 1.1688 - val_loss: 1.1052
Epoch 8/10
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1.1345 - val_loss: 1.0866
Epoch 9/10
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 1.1464 - val_loss: 1.0679
Epoch 10/10
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1.1024 - val_loss: 1.0496
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 731us/step


'Mean Absolute Difference: '

6.687161136412926

### Step 3
Repeat step 1 on this noisy dataset to obtain a new set of feature importance rankings.

In [12]:
# Given a single instance, get rankings for original and noisy data, then compare them (next steps).
feature_importances_noisy = FeatureImportanceCalculator(clf, X_train_noisy)

lime_ranking_noisy = feature_importances_noisy.get_lime_ranking(X_test_noisy.iloc[sample_idx])
shap_ranking_noisy = feature_importances_noisy.get_shap_ranking(X_test_noisy.iloc[sample_idx], explainer_type=shap.TreeExplainer)
anchor_ranking_noisy = feature_importances_noisy.get_anchor_ranking(X_test_noisy.iloc[sample_idx])

display(lime_ranking_noisy, shap_ranking_noisy, anchor_ranking_noisy)

,feature,score
0,Checking_account_none,-0.061149
1,Duration,0.052294
2,Checking_account_little,0.034383
3,Age,-0.024232
4,Checking_account_moderate,0.017083
5,Housing_own,-0.012519
6,Saving_accounts_little,0.009584
7,Sex_male,-0.008883
8,Sex_female,0.005627
9,Credit_amount,0.005627


,feature,score
0,Checking_account_none,-0.048790
1,Duration,0.047748
2,Checking_account_moderate,-0.037637
3,Housing_free,-0.032382
4,Age,0.030560
5,Credit_amount,0.029636
6,Checking_account_little,0.024052
7,Job_skilled,0.013991
8,Saving_accounts_moderate,0.010242
9,Sex_female,0.009589


,feature,score
0,Age,0.503277
1,Duration,0.433814
2,Checking_account_little,0.283093
3,Credit_amount,0.250328
4,Saving_accounts_rich,0.000000
5,Purpose_repairs,0.000000
6,Purpose_radio_TV,0.000000
7,Purpose_furniture_equipment,0.000000
8,Purpose_education,0.000000
9,Purpose_domestic_appliances,0.000000


### Step 4
Assess the robustness of each explanation model by comparing its feature importance ranking on the original data with the ranking on the noisy data.

In [13]:
def robustness_metrics(ranking_original: pd.DataFrame, ranking_noisy: pd.DataFrame) -> pd.DataFrame:
    """
    Returns a DataFrame with 4 robustness metrics of a given feature importance ranking:
        1. "mean_squared_differece": the mean squared difference between the scores of the original and noisy rankings; (previously called "stability") | Lower is better
        2. "mean_absolute_difference": the mean absolute difference between the scores of the original and noisy rankings; (previously called "sensitivity") | Lower is better
        3. "spearman_correlation": the Spearman correlation | Higher is better
        4. "pearson_correlation": the Pearson correlation | Higher is better
    """
    
    # Align dataframes:
    ranking_original = ranking_original.set_index('feature')
    ranking_noisy = ranking_noisy.set_index('feature')
    ranking_original = ranking_original.reindex(ranking_noisy.index)

    # Compute metrics:
    mean_squared_difference = ((ranking_original['score'] - ranking_noisy['score']) ** 2).mean()
    mean_absolute_difference = np.abs(ranking_original['score'] - ranking_noisy['score']).mean()
    spearman_correlation = spearmanr(ranking_original['score'], ranking_noisy['score']).correlation
    pearson_correlation = pearsonr(ranking_original['score'], ranking_noisy['score'])[0]

    robustness_metrics = pd.DataFrame({
        'mean_squared_difference': [mean_squared_difference],
        'mean_absolute_difference': [mean_absolute_difference],
        'spearman_correlation': [spearman_correlation],
        'pearson_correlation': [pearson_correlation]
    })

    return robustness_metrics

In [14]:
print("Robustness metrics for LIME:")
display(robustness_metrics(lime_ranking, lime_ranking_noisy))

print("Robustness metrics for SHAP:")
display(robustness_metrics(shap_ranking, shap_ranking_noisy))

print("Robustness metrics for Anchor:")
display(robustness_metrics(anchor_ranking, anchor_ranking_noisy))

Robustness metrics for LIME:


,mean_squared_difference,mean_absolute_difference,spearman_correlation,pearson_correlation
0,0.000007,0.002036,0.925123,0.989875


Robustness metrics for SHAP:


,mean_squared_difference,mean_absolute_difference,spearman_correlation,pearson_correlation
0,0.000092,0.005445,0.65558,0.863946


Robustness metrics for Anchor:


,mean_squared_difference,mean_absolute_difference,spearman_correlation,pearson_correlation
0,0.025003,0.036697,0.706849,0.574389


### Step 5

In [22]:
def explain_instance(instance_data_row):
    lime_ranking = feature_importances.get_lime_ranking(instance_data_row)
    lime_ranking_noisy = feature_importances_noisy.get_lime_ranking(instance_data_row)
    lime_instance_metrics = robustness_metrics(lime_ranking, lime_ranking_noisy)

    shap_ranking = feature_importances.get_shap_ranking(instance_data_row, explainer_type=shap.TreeExplainer)
    shap_ranking_noisy = feature_importances_noisy.get_shap_ranking(instance_data_row, explainer_type=shap.TreeExplainer)
    shap_instance_metrics = robustness_metrics(shap_ranking, shap_ranking_noisy)

    anchor_ranking = feature_importances.get_anchor_ranking(instance_data_row)
    anchor_ranking_noisy = feature_importances_noisy.get_anchor_ranking(instance_data_row)
    anchor_instance_metrics = robustness_metrics(anchor_ranking, anchor_ranking_noisy)

    ### Use toposis to assign weights to the explanation models:
    evaluation_matrix = np.array([
        lime_instance_metrics.values.flatten(),
        shap_instance_metrics.values.flatten(),
        anchor_instance_metrics.values.flatten()
    ])

    display(evaluation_matrix)

    robustness_metrics_weights = [
        0.25, # mean squared difference
        0.25, # mean absolute difference
        0.25, # spearman correlation
        0.25  # pearson correlation
    ]

    # if higher value is preferred - True
    # if lower value is preferred - False
    criterias = np.array([
        False,  # For mean_squared_difference, lower is better
        False,  # For mean_absolute_difference, lower is better
        True,   # For spearman_correlation, higher is better
        True    # For pearson_correlation, higher is better
    ])

    t = Topsis(evaluation_matrix, robustness_metrics_weights, criterias, debug=True)
    t.calc()

    display(t.best_similarity)
    # Sujoy suggested using the best_similarity as weights for the explanations

    lime_weight, shap_weight, anchor_weight = t.best_similarity
    result = lime_ranking.merge(shap_ranking, on='feature', how='outer', suffixes=('_lime', '_shap'))
    result = result.merge(anchor_ranking, on='feature', how='outer', suffixes=('_lime', '_shap'))
    result = result.rename(columns={'score': 'score_anchor'})
    result['score_ensemble'] = lime_weight * result['score_lime'] + shap_weight * result['score_shap'] + anchor_weight * result['score_anchor']
    result = result.sort_values(by='score_ensemble', ascending=False).reset_index(drop=True)

    return result


In [23]:
sample_idx = 0
explain_instance(X_test.iloc[sample_idx])

array([[0.00000482, 0.0017169 , 0.94137931, 0.99377704],
       [0.00000295, 0.00112749, 0.97634885, 0.99530053],
       [0.0402156 , 0.07321372, 0.41917808, 0.47940457]])

Step 1 - Evaluation Matrix:
 [[0.00000482 0.0017169  0.94137931 0.99377704]
 [0.00000295 0.00112749 0.97634885 0.99530053]
 [0.0402156  0.07321372 0.41917808 0.47940457]]

Step 2 - Normalized Evaluation Matrix:
 [[0.00011976 0.02344132 0.66314672 0.6687827 ]
 [0.00007347 0.01539395 0.68778072 0.66980796]
 [0.99999999 0.99960669 0.29528647 0.32262517]]

Step 3 - Weighted Normalized Evaluation Matrix
 [[0.00002994 0.00586033 0.16578668 0.16719567]
 [0.00001837 0.00384849 0.17194518 0.16745199]
 [0.25       0.24990167 0.07382162 0.08065629]]

Step 4 - worst_alternatives | best_alternatives 
 [0.25       0.24990167 0.07382162 0.08065629]  |  [0.00001837 0.00384849 0.17194518 0.16745199]

Step 5 - Distances to Worst Alternative | Distances to Best Alternative
 [0.37146715 0.37442584 0.        ] [0.00648386 0.         0.37442584]

Step 6 - Similarites to Worst Alternative | Similarities to Best Alternative
 [0.9828447 1.        0.       ] [0.0171553 0.        1.       ]



array([0.0171553, 0.       , 1.       ])

,feature,score_lime,score_shap,score_anchor,score_ensemble
0,Purpose_furniture_equipment,-0.003705,0.009065,0.813893,8.138290e-01
1,Duration,0.054372,0.051965,0.804718,8.056510e-01
2,Age,-0.025112,0.044427,0.503277,5.028457e-01
3,Checking_account_little,0.039987,0.030740,0.283093,2.837790e-01
4,Checking_account_moderate,0.018006,-0.025005,0.000000,3.088994e-04
5,Saving_accounts_little,0.011066,-0.003987,0.000000,1.898443e-04
6,Credit_amount,0.006880,0.018809,0.000000,1.180288e-04
7,Sex_female,0.006711,0.007021,0.000000,1.151211e-04
8,Housing_free,0.005453,0.006844,0.000000,9.354491e-05
9,Housing_rent,0.005364,0.000178,0.000000,9.201675e-05


### Pevious attempt at step 5 (IGNORE)